In [1]:
import numpy as np

In [2]:
import ufl

In [3]:
from dolfinx import fem, io, mesh, plot

[cppreviewdu-dolfinxdemo-fe7nn62qbto:00348] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.cppreviewdu-dolfinxdemo-fe7nn62qbto.33333/jf.0/3889496064/shared_mem_cuda_pool.cppreviewdu-dolfinxdemo-fe7nn62qbto could be created.
[cppreviewdu-dolfinxdemo-fe7nn62qbto:00348] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [4]:
from dolfinx.fem import petsc

In [5]:
from ufl import ds, dx, grad, inner

In [6]:
from mpi4py import MPI

In [7]:
from petsc4py.PETSc import ScalarType
from dolfinx.fem.petsc import LinearProblem

In [8]:
msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0, 0.0), (2.0, 1.0)),
    n=(32, 16),
    cell_type=mesh.CellType.triangle,
)

In [10]:
V = fem.functionspace(msh, ("Lagrange", 1))

In [11]:
facets = mesh.locate_entities_boundary(
    msh,
    dim=1,
    marker=lambda x: np.logical_or(np.isclose(x[0], 0.0), np.isclose(x[0], 2.0)),
)

In [12]:
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

In [13]:
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [14]:
u = ufl.TrialFunction(V)

In [15]:
v = ufl.TestFunction(V)

In [16]:
x = ufl.SpatialCoordinate(msh)

In [17]:
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.002)

In [18]:
g = ufl.sin(5 * x[0])

In [19]:
a = inner(grad(u), grad(v)) * dx

In [20]:
L = inner(f, v) * dx + inner(g, v) * ds

In [21]:
# precondicionadores lu, gmrs (depende de la forma de la matriz)
problem = LinearProblem(
    a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}
)

In [22]:
uh = problem.solve()

In [23]:
with io.VTKFile(msh.comm, "out_poisson/poisson.pvd", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)

In [24]:
!(ls out_poisson)

poisson.pvd	    poisson000001.pvtu	   poisson_p0_000001.vtu
poisson000000.pvtu  poisson_p0_000000.vtu


In [25]:
try:
    import pyvista

    cells, types, x = plot.vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uh.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    warped = grid.warp_by_scalar()
    plotter.add_mesh(warped)
    if pyvista.OFF_SCREEN:
        pyvista.start_xvfb(wait=0.1)
        plotter.screenshot("uh_poisson.png")
    else:
        plotter.show()
except ModuleNotFoundError:
    print("pyvista is required")